In [ ]:
%pip install numpy tifffile matplotlib scikit-image scipy "napari[all]" pyocclient

In [ ]:
import numpy as np
import tifffile
import matplotlib.pyplot as plt
from skimage.registration import phase_cross_correlation
from scipy.ndimage import shift
import napari

In [ ]:
import owncloud
import os

if not os.path.exists('data'):
    print('Creating directory for data')
    os.mkdir('data')

if not os.path.exists('data/data.tif'):
    oc = owncloud.Client.from_public_link('https://uni-bonn.sciebo.de/s/bFDLSfaxRqKlqT7')
    oc.get_file('/', 'data/data.tif');

In [ ]:
movie = tifffile.imread('data/data.tif')
movie.shape

For every frame, compute how much it has to be shifted relative to every other frame. Because computation time increases exponentially with the number of frames, we only do this for a subset.

In [ ]:
frames = movie[0:100]
n_frames = frames.shape[0]

In [ ]:
shift_map = np.zeros((n_frames, n_frames))
for i in range(n_frames):
    for j in range(n_frames):
        shift = phase_cross_correlation(frames[i], frames[j])[0]
        shift_map[i,j] = np.abs(shift).sum()

We can plot the resulting matrix as an image. Each row and column is an image and a higher value indicates a larger shift required two align a given pair of images.

In [ ]:
im = plt.imshow(shift_map)
plt.colorbar(im)

The diagonal of this matrix is 0 because we don't need to align an image with itself

In [ ]:
np.diag(shift_map)

Now, we can average the matrix across one dimension and find the index where this is minimal. This is the image that, on average, needs the least shifting

In [ ]:
np.argmin(shift_map.mean(axis=0))